Team Members:


*   Vrinda Sharma
*   Vinay Locharulu
*   Tanvi Modi
*   Shayna Patel
*   Stanley Wen
*   Yunwei Yan


In [7]:
import requests
import csv
from bs4 import BeautifulSoup, NavigableString, Tag
import logging
import os
import pandas as pd
# from google.colab import files
import numpy as np


# TASK A - Scraper

In [0]:
url = 'https://www.beeradvocate.com/beer/top-rated/'
page = requests.get(url)
page_html = page.text
soup_main = BeautifulSoup(page_html, 'html.parser')
beers = soup_main.find_all(class_ = "hr_bottom_light")

In [0]:
beers_df = pd.DataFrame(columns = ['product_name', 'product_review', 'user_rating'])

In [0]:
for beer in beers[1::5]:
    beerinfo = beer.find('a')
    beer_link = beer.find('a')['href']
    link = 'https://www.beeradvocate.com'+ str(beer_link)
    beer_page = requests.get(link)
    beer_html = beer_page.text

    soup = BeautifulSoup(beer_html, 'html.parser')

    reviews = soup.find_all(id = 'rating_fullview_content_2')

    for review in reviews:
        product_name = beerinfo.string
        user_rating = review.find(class_ = 'BAscore_norm').string
        
        for br in review.find_all('br'):
            if isinstance(br.nextSibling,NavigableString):
                product_review = br.nextSibling
 
        row = [{'product_name': product_name, 'product_review':product_review, 'user_rating':user_rating}]
        beers_df = beers_df.append(row)

    
        

In [0]:
beers_df.head(10)

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,"2016 Silver Wax. Aroma has whiskey, maple, tof...",4.8
0,Kentucky Brunch Brand Stout,The beer pours Pitch Black with a frothy tan h...,4.74
0,Kentucky Brunch Brand Stout,Probably the smoothest beer I have ever had. S...,4.68
0,Kentucky Brunch Brand Stout,"Dark black, very thick, a little bit of tan he...",5
0,Kentucky Brunch Brand Stout,Poured black as ink with thin ruby edges at 58...,4.97
0,Kentucky Brunch Brand Stout,she’s got heat....but man is she somethin nice...,4.75
0,Kentucky Brunch Brand Stout,Amazing brew. The maple aroma pours out of thi...,5
0,Kentucky Brunch Brand Stout,Finally got to try the white whale. Pours like...,4.3
0,Kentucky Brunch Brand Stout,Had this on tap at the 12-15-18 release in Dec...,4.85
0,Kentucky Brunch Brand Stout,Had this on tap At the KBBS release 12/15/18 p...,4.89


In [0]:
beers_df.to_csv('Beer_Reviews.csv')
files.download('Beer_Reviews.csv')

In [9]:
beers_df = pd.read_csv('Beer_Reviews.csv')
beers_df.dropna(inplace = True)
beers_df['product_review']

0       2016 Silver Wax. Aroma has whiskey, maple, tof...
1       The beer pours Pitch Black with a frothy tan h...
2       Probably the smoothest beer I have ever had. S...
3       Dark black, very thick, a little bit of tan he...
4       Poured black as ink with thin ruby edges at 58...
5       she’s got heat....but man is she somethin nice...
6       Amazing brew. The maple aroma pours out of thi...
7       Finally got to try the white whale. Pours like...
8       Had this on tap at the 12-15-18 release in Dec...
9       Had this on tap At the KBBS release 12/15/18 p...
10      Great beer, but didn't seem to be worth the hy...
11           Batch one, 2013, Enjoyed at GAB bottle share
12      Pouring with a thick viscosity that's damn nea...
13      This is the beer that I always thought Morning...
14      A buddy of mine called me one night and said t...
15      its always an occasion to drink what is deemed...
16      Love the huge maple up front with balanced whi...
17      Jet bl

# PART B

#   We chose below attributes :
    *  hoppy
    *  fruity
    *  complex


In [14]:
from string import punctuation
def remove_punctuation(item):
    for p in punctuation: 
        item = item.lstrip().replace(p,'')
    return item

def conv_lowercase(x):
    return x.lower()

beers_df['clean_reviews'] = beers_df['product_review'].apply(remove_punctuation).apply(conv_lowercase)


# beers_df['clean_reviews']

import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

beers_df['list_review'] = beers_df['clean_reviews'].apply(word_tokenize).apply(set).apply(list)

# Remove stop words
stop_words = set(stopwords.words('english'))
def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 

beers_df['final_review'] = beers_df['list_review'].apply(remove_stopwords)

# Creating a list of all the words in the comments to count word frequency in the next step
count = []
for i in range(len(beers_df)):
    count+=beers_df['final_review'][i]

word_list = []
for i in range(len(beers_df)):
    word_list+=beers_df['final_review'][i]

# fetching the frequency of words
from nltk import FreqDist
word_freq = nltk.FreqDist(word_list)
word_freq

attributes = ['fruity', 'complex','hoppy']

comp = word_freq['complex']
fruity = word_freq['fruity']
hoppy = word_freq['hoppy']

print(comp, fruity,hoppy)

206 148 94


In [10]:
beers_df.head()

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,"2016 Silver Wax. Aroma has whiskey, maple, tof...",4.80
1,Kentucky Brunch Brand Stout,The beer pours Pitch Black with a frothy tan h...,4.74
2,Kentucky Brunch Brand Stout,Probably the smoothest beer I have ever had. S...,4.68
3,Kentucky Brunch Brand Stout,"Dark black, very thick, a little bit of tan he...",5.00
4,Kentucky Brunch Brand Stout,Poured black as ink with thin ruby edges at 58...,4.97


In [0]:
!pip install spacy
!python -m spacy download en_core_web_md
!python -m spacy download de_core_news_sm

     |████████████████████████████████| 95.4MB 1.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126237 sha256=bd9709df7d68294e14fd639d52b6c5f468a425d2769738ac745208f7ba319ed6
  Stored in directory: /tmp/pip-ephem-wheel-cache-f5g6iajf/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
     |████████████████████████████████| 11.1MB 1.7MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.1.0-cp36-none-any.whl size=11073065 sha256=927dd103bad23bb9249655e66a0588631b7f93e19d831226062d57b59046ee28
  Stored in directory: /tmp/pip-ephem-wheel-cache-msz8ksav/wheels/b4/8b/5e/d2ce5d2756ca95de22f50f68299708009a4aafda2aea79c4e4
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


# PART C - Similarity Analysis

In [12]:
import spacy 
# !python -m spacy download en
nlp = spacy.load('en_core_web_md\\en_core_web_md-2.2.0') 
# nlp = spacy.load('en_core_web_sm')



In [15]:
beers_df['clean_reviews'].head()

0    2016 silver wax aroma has whiskey maple toffee...
1    the beer pours pitch black with a frothy tan h...
2    probably the smoothest beer i have ever had sm...
3    dark black very thick a little bit of tan head...
4    poured black as ink with thin ruby edges at 58...
Name: clean_reviews, dtype: object

### PART - C

In [16]:
check_list = []
prod_name=[]
index = []

for i in range(len(beers_df)):
    check =  any(item in attributes for item in beers_df['final_review'][i])

    if check is True:
        check_list.append(beers_df['clean_reviews'][i])
        prod_name.append(beers_df['product_name'][i])
        index.append(i)

In [17]:
df_init_output = pd.DataFrame(prod_name,columns =['product_name'])
df_init_output['product_review'] = check_list
df_init_output['index'] = index
df_init_output.head()

,product_name,product_review,index
0,Hunahpu's Imperial Stout - Double Barrel Aged,tasted from bottle pours black with beige head...,56
1,Hunahpu's Imperial Stout - Double Barrel Aged,pours a thick oily pitch black coloration with...,59
2,Hunahpu's Imperial Stout - Double Barrel Aged,wow the nose of this is just stupid one of the...,72
3,Barrel-Aged Abraxas,pours a pitch black coloration with a small cr...,79
4,Barrel-Aged Abraxas,this is worth the hype the aroma is off the ch...,94


In [18]:
# Percentage similarity of attributes and each review that contains any of the attributes 

'''file_name = 'attribute_file.txt'

f=open(file_name, "r")
if f.mode == 'r':
    contents =f.read()'''
contents = 'fruity, complex,hoppy'
doc1 = nlp ('fruity, complex,hoppy')
similarity=[]
for j in range(len(df_init_output['product_review'])):
    doc2 = nlp(df_init_output['product_review'][j])
    similarity.append(doc1.similarity(doc2))
    print("Percentage similarity of "+contents+" with each review is : ",doc1.similarity(doc2))

Percentage similarity of fruity, complex,hoppy with each review is :  0.7460392152977046
Percentage similarity of fruity, complex,hoppy with each review is :  0.6785271774992633
Percentage similarity of fruity, complex,hoppy with each review is :  0.6546107812381365
Percentage similarity of fruity, complex,hoppy with each review is :  0.7746767724055973
Percentage similarity of fruity, complex,hoppy with each review is :  0.6724718513578966
Percentage similarity of fruity, complex,hoppy with each review is :  0.7160818212685623
Percentage similarity of fruity, complex,hoppy with each review is :  0.7952228394082664
Percentage similarity of fruity, complex,hoppy with each review is :  0.5752725128739881
Percentage similarity of fruity, complex,hoppy with each review is :  0.5798123222883477
Percentage similarity of fruity, complex,hoppy with each review is :  0.7474566780151772
Percentage similarity of fruity, complex,hoppy with each review is :  0.678139783307387
Percentage similarity 

Percentage similarity of fruity, complex,hoppy with each review is :  0.741957854964022
Percentage similarity of fruity, complex,hoppy with each review is :  0.6915947817637232
Percentage similarity of fruity, complex,hoppy with each review is :  0.7184603169286008
Percentage similarity of fruity, complex,hoppy with each review is :  0.6927895025894547
Percentage similarity of fruity, complex,hoppy with each review is :  0.678064561046685
Percentage similarity of fruity, complex,hoppy with each review is :  0.6695364059500101
Percentage similarity of fruity, complex,hoppy with each review is :  0.6797995372765472
Percentage similarity of fruity, complex,hoppy with each review is :  0.6438005720600058
Percentage similarity of fruity, complex,hoppy with each review is :  0.7547557839729304
Percentage similarity of fruity, complex,hoppy with each review is :  0.7111980116856875
Percentage similarity of fruity, complex,hoppy with each review is :  0.5971453174375518
Percentage similarity o

Percentage similarity of fruity, complex,hoppy with each review is :  0.7070970358047376
Percentage similarity of fruity, complex,hoppy with each review is :  0.7327041527325125
Percentage similarity of fruity, complex,hoppy with each review is :  0.6343383276334981
Percentage similarity of fruity, complex,hoppy with each review is :  0.7504223954837306
Percentage similarity of fruity, complex,hoppy with each review is :  0.7059252625056414
Percentage similarity of fruity, complex,hoppy with each review is :  0.6630835647435152
Percentage similarity of fruity, complex,hoppy with each review is :  0.6999420374642586
Percentage similarity of fruity, complex,hoppy with each review is :  0.6135350341757331
Percentage similarity of fruity, complex,hoppy with each review is :  0.7726932657874745
Percentage similarity of fruity, complex,hoppy with each review is :  0.7144589572292513
Percentage similarity of fruity, complex,hoppy with each review is :  0.6985606728875235
Percentage similarity

In [28]:
df_output = df_init_output
df_output['similarity_scores'] = similarity
df_top_300 = df_output.sort_values(by='similarity_scores', ascending=False)[:300]
df_top_300.reset_index(drop=True,inplace=True)

df_top_300.iloc[:,[0,1,3]].to_csv('similarity_analysis.csv',index=False)
df_top_300.head()

,product_name,product_review,index,similarity_scores
0,Duck Duck Gooze,on tap at beavertown extravaganza aroma has so...,732,0.800673
1,Heady Topper,golden with slight amber hue good foamy head a...,123,0.795223
2,Galaxy Dry Hopped Fort Point Pale Ale,112014 wow this is amazing incredible aromas ...,2063,0.790210
3,Barrel-Aged Abraxas,pours a pitch black coloration with a small cr...,79,0.774677
4,Bourbon Barrel Oro Negro,black color with mocha head aroma has intense ...,5487,0.772693


In [30]:
df_top_300['similarity_scores'] = df_top_300['similarity_scores'].astype('float')
table = pd.pivot_table(df_top_300, values='similarity_scores', index='product_name', aggfunc=np.mean)
recomm_table =table.sort_values(by=['similarity_scores'], ascending=False)

In [31]:
recomm_table

,similarity_scores
product_name,
Flora Plum,0.772557
Pseudo Sue,0.771635
Double Galaxy,0.766673
Barrel-Aged Silhouette,0.765579
Aurelian Lure,0.756973
Heady Topper,0.755652
Scaled Way Up,0.748234
Zombie Dust,0.747394
Sang Noir,0.746651


# PART D - Sentiment Analysis

In [32]:
! pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np

In [33]:
beers_df = pd.read_csv('Beer_Reviews.csv')

In [34]:
top_beers = beers_df.loc[df_top_300['index'],['product_name','product_review']]
top_beers

,product_name,product_review
732,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...
123,Heady Topper,Golden with slight amber hue. Good foamy head ...
2063,Galaxy Dry Hopped Fort Point Pale Ale,"11/2014 - Wow, this is amazing; incredible aro..."
79,Barrel-Aged Abraxas,"Pours a pitch black coloration with a small, c..."
5487,Bourbon Barrel Oro Negro,Black color with mocha head. Aroma has intense...
2037,Flora Plum,"insanely tart and funky, bright acidity, lemon..."
2980,Pseudo Sue,Murky creamsicle orange in color. Poured with ...
1212,Focal Banger,"Poured from can to tulip, hazy yellow, white h..."
5359,Hop JuJu Imperial IPA,Bright copper orange in color with perfect cla...
5517,Juicy,Hazy orange color with off-white head. Aroma h...


In [35]:
top_beers['senti_pos'] = ""
top_beers['senti_neu'] = ""
top_beers['senti_neg'] = ""
top_beers['sentiment'] = ""

In [36]:
sentimentlist=[]
analyzer = SentimentIntensityAnalyzer()
for index, row in top_beers.iterrows():
    vs = analyzer.polarity_scores(row['product_review'])
    sentimentlist.append(vs['compound'])
    top_beers.loc[index, 'senti_pos'] = vs['pos']
    top_beers.loc[index, 'senti_neu'] = vs['neu']
    top_beers.loc[index, 'senti_neg'] = vs['neg']
    top_beers.loc[index, 'sentiment'] = vs['compound']

In [37]:
top_beers

,product_name,product_review,senti_pos,senti_neu,senti_neg,sentiment
732,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...,0.151,0.849,0,0.6908
123,Heady Topper,Golden with slight amber hue. Good foamy head ...,0.23,0.695,0.075,0.7003
2063,Galaxy Dry Hopped Fort Point Pale Ale,"11/2014 - Wow, this is amazing; incredible aro...",0.389,0.611,0,0.9396
79,Barrel-Aged Abraxas,"Pours a pitch black coloration with a small, c...",0.166,0.811,0.023,0.9366
5487,Bourbon Barrel Oro Negro,Black color with mocha head. Aroma has intense...,0.075,0.925,0,0.25
2037,Flora Plum,"insanely tart and funky, bright acidity, lemon...",0.211,0.694,0.095,0.8171
2980,Pseudo Sue,Murky creamsicle orange in color. Poured with ...,0.175,0.743,0.081,0.9286
1212,Focal Banger,"Poured from can to tulip, hazy yellow, white h...",0,0.922,0.078,-0.4215
5359,Hop JuJu Imperial IPA,Bright copper orange in color with perfect cla...,0.11,0.813,0.077,0.1545
5517,Juicy,Hazy orange color with off-white head. Aroma h...,0.061,0.843,0.096,-0.204


In [38]:
sort_df = top_beers.sort_values(by=['sentiment'], ascending=False)
sort_df

,product_name,product_review,senti_pos,senti_neu,senti_neg,sentiment
3504,Darkness,"750 ml bottle into snifter, bottled on 9/25/20...",0.21,0.731,0.059,0.9976
5532,Juicy,an extremely impressive little bottle of saiso...,0.193,0.783,0.024,0.9975
3081,Samuel Adams Utopias,8000th review! 8 ounce pour from a 24 ounce bo...,0.173,0.78,0.047,0.9973
5673,Bourbon Barrel Champion Ground,bourbon barrel aged imperial stout with coffee...,0.227,0.73,0.043,0.9964
4508,La Fosse,i am surprised to learn this is the same base ...,0.199,0.77,0.031,0.996
3263,Mocha Wednesday,"750 ml bottle into snifter, bottled on 1/27/20...",0.169,0.796,0.035,0.9959
3881,Upper Case,"Outstanding beer, and very different. Actually...",0.266,0.734,0,0.9957
1617,Beer:Barrel:Time,It poured a super thick and viscous in the gla...,0.305,0.662,0.033,0.9954
1333,Lou Pepe - Framboise,a 2010 vintage generously shared with me after...,0.193,0.783,0.024,0.9938
4738,Gueuze 100% Lambic,"2013 variant, somewhat gentle but down the mid...",0.337,0.638,0.025,0.9926


In [40]:
sort_df['sentiment'] = sort_df['sentiment'].astype('float')
table = pd.pivot_table(sort_df, values='sentiment', index='product_name', aggfunc=np.mean)
sentim_table =table.sort_values(by=['sentiment'], ascending=False)

In [41]:
sentim_table

,sentiment
product_name,
Bourbon Barrel Champion Ground,0.996400
La Fosse,0.996000
Mocha Wednesday,0.995900
Barrel-Aged Silhouette,0.984700
Coffee Abraxas,0.984300
Genealogy Of Morals - Bourbon Barrel-Aged,0.983200
Tweak,0.981100
Coolship Red,0.978600
Beer:Barrel:Time,0.974733


# PART E - Recommendations

In [42]:
results = recomm_table.join(sentim_table, on='product_name', how='outer', lsuffix="_review")
results['scores']= results['sentiment']+results['similarity_scores']
results= results.sort_values('scores',ascending=False)
results.head(3)

,similarity_scores,sentiment,scores
product_name,,,
Barrel-Aged Silhouette,0.765579,0.9847,1.750279
Mocha Wednesday,0.745125,0.9959,1.741025
Genealogy Of Morals - Bourbon Barrel-Aged,0.740421,0.9832,1.723621


# PART F - Analysis

In [43]:
beer_avg_ratings = beers_df[['product_name', 'user_rating']].copy()

In [44]:
grouped_beers = beer_avg_ratings.groupby('product_name').mean()

In [45]:
sorted_beers = grouped_beers.sort_values('user_rating', ascending=False)

In [46]:
#these are the top three beers without taking similarity or sentiment into account
print("Top three beers before similarity and sentiment analysis")
sorted_beers.head(3)

Top three beers before similarity and sentiment analysis


,user_rating
product_name,
Kentucky Brunch Brand Stout,4.812000
Chemtrailmix,4.811176
Barrel-Aged Abraxas,4.798000




*  None of these are the same as top three recommended
*  Should check if they are in top 10 recommded 
*  Maybe compare sentiment score and user rating 



In [47]:
print("Top ten recommended beers to our customer")
results.head(10)

Top ten recommended beers to our customer


,similarity_scores,sentiment,scores
product_name,,,
Barrel-Aged Silhouette,0.765579,0.98470,1.750279
Mocha Wednesday,0.745125,0.99590,1.741025
Genealogy Of Morals - Bourbon Barrel-Aged,0.740421,0.98320,1.723621
Pseudo Sue,0.771635,0.92860,1.700235
§ucaba,0.741958,0.95510,1.697058
Abrasive Ale,0.740907,0.95170,1.692607
Zombie Dust,0.747394,0.94120,1.688594
Tweak,0.706910,0.98110,1.688010
Double Nelson,0.732110,0.94515,1.677260


Neither of the overall top three beers were included in our top ten list of recommended beers. If we were to recommend these to our customer they would be dissatisfied, proving the need for similarity and sentiment analysis. This also shows that ratings for beers may not be completely relevant for each user. In this example, we took 25 reviews from each beer but in reality we  have to take caution that we don't weigh one rating too heavily. For example, if a beer has only one positive review it will rank high on our list but that review may not be reflective of the overall sentiment towards the beer. 

Barrel-Aged Abraxas was the only beer from the overall top three to be included in our recommended beers list made from the top 300 beers based on the reviews. This further proves that accurate recomendations cannot be made based only from the general ratings. 